Extract S&P 500 stocks' names on wikipedia

In [139]:
import pywikibot
import numpy as np
import re
import time
import datetime
import pandas as pd
import yfinance as yf


In [ ]:
site = pywikibot.Site('en', 'wikipedia')
page = pywikibot.page.BasePage(site, 'List of S&P 500 companies')
text = page.text


pattern = re.compile(r'(?:NyseSymbol|NasdaqSymbol)\|[A-Z]{1,6}')
result = pattern.findall(text)
names = [re.sub('(?:NyseSymbol|NasdaqSymbol)\|', '',c) for c in result]
names   


Calculate monthly return & risk(standard deviation) and save to sp_500.csv

In [141]:
start = datetime.datetime(2016,10,1)
end = datetime.datetime(2017,10,1)
sp_500 = pd.DataFrame(columns = ['Return', 'Risk'], index = names)  


Function to calculate mean(expected return) and standard deviation(risk) of stocks

In [142]:
def calculate_stock(name, start, end, period):
    try:
        prices = yf.download(name, start, end)
    except:
        return None
    
    Endings = prices.Close
    for i in range(len(Endings)):
        if Endings[i] is np.nan: Endings[i] = Endings[i-1]
    
    
    increases = []
    for i in range(len(Endings) - period):
        increase = (Endings[i+period] - Endings[i])/Endings[i]
        increases.append(increase)
    increases_ = np.array(increases)
    return (np.mean(increases_), np.std(increases_))

Storing data into dataframe

In [ ]:

for name in names:
    new_stock = calculate_stock(name, start, end, 20)
    sp_500.loc[name] = [new_stock[0], new_stock[1]]



In [144]:
mod_500 = sp_500.drop(['BRK','MRNA','DVN','FANG','OXY','TRGP'])#droped due to unusual behaviour
mod_500

,Return,Risk
MMM,0.018537,0.030199
AOS,0.016322,0.047595
ABT,0.021523,0.046717
ABBV,0.030361,0.066345
ACN,0.013339,0.038104
...,...,...
XYL,0.021732,0.043953
YUM,0.016765,0.036767
ZBRA,0.042692,0.073353
ZBH,-0.004918,0.081922


## Choose 50 best stocks from S&P500.

In [145]:
np.sum(mod_500.Return > mod_500.Risk*0.65)

56

In [146]:
sp_max51 = mod_500[mod_500.Return > mod_500.Risk*0.65]

In [147]:
sp_max51.to_csv('SP500_preparation.csv')

In [148]:
sp_max51

,Return,Risk
ADBE,0.030493,0.037011
A,0.032049,0.034831
ALB,0.039994,0.059843
ALGN,0.06653,0.062775
ALL,0.024929,0.033204
AXP,0.02887,0.040548
AME,0.031155,0.038887
APH,0.020998,0.024724
AON,0.023201,0.025769
AMAT,0.044509,0.057492
